In [2]:
import numpy as np
import matplotlib.pyplot as plt
import random
from numba import njit
from sklearn.cluster import KMeans

N=1024
var_J=1

@njit
def initial_lattice(N):
    M=np.ones((N))
    l=np.array([1,-1])
    for i in range(N):
            M[i]=np.random.choice(l)
    return(M)

@njit
def intraction(N,mean_J,var_J):
    J_matrix=np.zeros((N,N))
    for i in range(N):
        for j in range(N):
            if i<j :
                J_matrix[i][j]=random.gauss(mean_J/N,var_J/np.sqrt(N))
            else :
                J_matrix[i][j]=0
    J_matrix=J_matrix+np.transpose(J_matrix)  
    return(J_matrix)

@njit
def energy_local(M,J_matrix,i):
    H=np.sum(-J_matrix[i]*M[i]*M)
    return(H)

@njit
def energy_total(M,J_matrix,N):
    E_t=0
    for i in range(N):
        E_t=E_t+energy_local(M,J_matrix,i)
    E_t=E_t/2
    return(E_t)


@njit
def trial(J_matrix,N,T): 
    field_list=np.zeros(N)
    M=initial_lattice(N)
    E_t=energy_total(M,J_matrix,N)
    for trial in range(int(np.sqrt(N**3))):
        random_i=random.randrange(N)                
        M[random_i]=M[random_i]*(-1)
        delta_E=2*energy_local(M,J_matrix,random_i)
        if delta_E>0:
            p=np.exp(-delta_E/T)
            coin=random.random()
            if coin>p :
                M[random_i]=M[random_i]*(-1)
            else :
                E_t=E_t+delta_E
        else :
            E_t=E_t+delta_E
    
    for i in range(N):
        field=np.sum(-J_matrix[i]*M)
        field_list[i]=field

    return(field_list)

#spin glass
l_feature_SG=[]
for mean_J in np.arange(0.000001,0.010001,0.001):
    for T in np.arange(0.000001,0.010001,0.001):
        l=[]
        J_matrix=intraction(N,mean_J,var_J)
        field_time_series=trial(J_matrix,N,T)
        l.append(abs(np.mean(field_time_series)))
        l.append(np.var(field_time_series))
        l_feature_SG.append(l)

#paramagnetic  
l_feature_P=[]
for mean_J in np.arange(0.000001,0.010001,0.001):
    for T in np.arange(1000,1000.01,0.001):
        l=[]
        J_matrix=intraction(N,mean_J,var_J)
        field_time_series=trial(J_matrix,N,T)
        l.append(abs(np.mean(field_time_series)))
        l.append(np.var(field_time_series))
        l_feature_P.append(l)

#ferro    
l_feature_F=[]
for mean_J in np.arange(1000,1000.01,0.001):
    for T in np.arange(0.000001,0.010001,0.001):
        l=[]
        J_matrix=intraction(N,mean_J,var_J)
        field_time_series=trial(J_matrix,N,T)
        l.append(abs(np.mean(field_time_series))/mean_J)
        l.append(np.var(field_time_series))
        l_feature_F.append(l)

In [6]:
centroid_SG = np.mean(l_feature_SG, axis=0)
distances_SG = np.linalg.norm(l_feature_SG - centroid_SG, axis=1)
average_distance_SG = np.mean(distances_SG)

centroid_P = np.mean(l_feature_P, axis=0)
distances_P = np.linalg.norm(l_feature_P - centroid_P, axis=1)
average_distance_P = np.mean(distances_P)

centroid_F = np.mean(l_feature_F, axis=0)
distances_F = np.linalg.norm(l_feature_F - centroid_F, axis=1)
average_distance_F = np.mean(distances_F)

print("Centroid_SG:", centroid_SG)
print("Average distance to centroid SG:", average_distance_SG)

print("Centroid_P:", centroid_P)
print("Average distance to centroid P:", average_distance_P)

print("Centroid_F:", centroid_F)
print("Average distance to centroid F:", average_distance_F)

Centroid_SG: [0.037744   2.43309432]
Average distance to centroid SG: 0.07555923468902921
Centroid_P: [0.0254929  0.99835798]
Average distance to centroid P: 0.041774436907356674
Centroid_F: [0.9990208  1.00271863]
Average distance to centroid F: 0.03892498478496885
